<a href="https://colab.research.google.com/github/Agil0509/Machine-Learning/blob/main/Imbalanced_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [5]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [7]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [8]:
X = df.drop('Exited',axis = 1)
y = df['Exited'].copy()

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [10]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder

cat_feat = X_train.select_dtypes(exclude = [np.number]).columns
num_feat = X_train.select_dtypes(include = [np.number]).columns

In [11]:
cat_pipeline = Pipeline([
    ('impute',SimpleImputer(strategy ='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown = 'ignore'))
])

num_pipeline =Pipeline([
   ('impute', SimpleImputer(strategy = 'median')),('scaler',StandardScaler())
])

transformer = ColumnTransformer([
    ('cat',cat_pipeline,cat_feat),
    ('num',num_pipeline,num_feat)
],remainder= 'passthrough')

In [55]:
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

In [13]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(random_state=42)
rnd_clf.fit(X_train_transformed,y_train)

RandomForestClassifier(random_state=42)

In [14]:
rnd_clf.score(X_test_transformed,y_test)

0.855

In [15]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

voting_clf = VotingClassifier(
    estimators=[
        ('lr',LogisticRegression(random_state=42)),
        ('rf',RandomForestClassifier(random_state=42)),
        ('svc',SVC(random_state = 42))
    ]
)

In [16]:
voting_clf.fit(X_train_transformed,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [17]:
for name, clf in voting_clf.named_estimators_.items():
  print(name,'= ',clf.score(X_test_transformed,y_test))

lr =  0.809
rf =  0.855
svc =  0.8565


In [18]:
voting_clf.voting = "soft"
voting_clf.named_estimators['svc'].probability = True
voting_clf.fit(X_train_transformed,y_train)
voting_clf.score(X_test_transformed,y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.856

In [19]:
y_pred = voting_clf.predict(X_test_transformed)

In [20]:
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
accuracy_score(y_test,y_pred)

0.856

In [21]:
f1_score(y_test,y_pred)

0.4965034965034965

In [22]:
precision_score(y_test,y_pred).round(2)


0.79

In [23]:
recall_score(y_test,y_pred)


0.361323155216285

In [24]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

 # Resampling (Undersampling and Oversampling)

In [25]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [26]:
oversample = RandomOverSampler(sampling_strategy='minority')
X_train_over,y_train_over = oversample.fit_resample(X_train_transformed,y_train)

In [27]:
y_train.value_counts()

Exited
0    6356
1    1644
Name: count, dtype: int64

In [28]:
y_train_over.value_counts()

Exited
0    6356
1    6356
Name: count, dtype: int64

In [29]:
rnd_clf.fit(X_train_over,y_train_over)

RandomForestClassifier(random_state=42)

In [30]:
rnd_clf.score(X_test_transformed,y_test)

0.855

In [31]:
y_new = rnd_clf.predict(X_test_transformed)

In [32]:
f1_score(y_test,y_new)

0.5565749235474007

In [33]:
recall_score(y_test,y_new)

0.4631043256997455

In [34]:
precision_score(y_test,y_new)

0.6973180076628352

In [35]:
voting_clf.fit(X_train_over,y_train_over)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [36]:
for name,clf in voting_clf.named_estimators_.items():
  print(name,'=',clf.score(X_test_transformed,y_test))

lr = 0.725
rf = 0.855
svc = 0.8085


In [37]:
voting_clf.score(X_test_transformed,y_test)

0.833

In [38]:
y_new1 = voting_clf.predict(X_test_transformed)

In [39]:
f1_score(y_test,y_new1)

0.5926829268292683

# SMOTE

In [40]:
from imblearn.over_sampling import SMOTE

In [41]:
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_smote,y_train_smote = smote.fit_resample(X_train_transformed,y_train)

In [42]:
y_train_smote.value_counts()

Exited
0    6356
1    6356
Name: count, dtype: int64

In [43]:
rnd_clf.fit(X_train_smote,y_train_smote)

RandomForestClassifier(random_state=42)

In [44]:
rnd_clf.score(X_test_transformed,y_test)

0.857

In [45]:
y_smote = rnd_clf.predict(X_test_transformed)
f1_score(y_test,y_smote)

0.5517241379310345

In [46]:
voting_clf.fit(X_train_smote,y_train_smote)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [47]:
for name ,clf in voting_clf.named_estimators_.items():
  print(name,'=',clf.score(X_test_transformed,y_test))

lr = 0.7275
rf = 0.857
svc = 0.8085


In [48]:
voting_clf.score(X_test_transformed,y_test)

0.8385

In [49]:
y_smote1 = voting_clf.predict(X_test_transformed)

In [50]:
f1_score(y_test,y_smote1)

0.5937106918238995

# BalancedBaggingClassifier

In [51]:
from imblearn.ensemble import BalancedBaggingClassifier

base_classifier  = RandomForestClassifier(random_state = 42)
balanced_bagging_clf = BalancedBaggingClassifier(base_classifier,
                                                sampling_strategy = 'auto',
                                                replacement = False,
                                                random_state = 42)
balanced_bagging_clf.fit(X_train_transformed,y_train)

BalancedBaggingClassifier(estimator=RandomForestClassifier(random_state=42),
                          random_state=42)

In [52]:
balanced_bagging_clf.score(X_test_transformed,y_test)

0.821

In [53]:
y_balanced = balanced_bagging_clf.predict(X_test_transformed)

In [54]:
f1_score(y_test,y_balanced)

0.6091703056768559